In [96]:
import pandas as pd
import pyarrow as pa
import numpy as np
import json
from datetime import date, datetime

In [116]:
def parseDate(date):
    if date is None:
        return date
    
    try:
        return datetime.strptime(date, '%d-%b-%y')
    except:
        try:
            return datetime.strptime(date, '%Y-%m-%d')
        except:
            return datetime.strptime(date, '%B, %Y')

In [117]:
data = pd.read_json('movies.json', encoding='utf-8')
data.describe()

,IMDB_Rating,IMDB_Votes,Production_Budget,Rotten_Tomatoes_Rating,Running_Time_min,US_DVD_Sales,US_Gross,Worldwide_Gross
count,2988.000000,2988.000000,3.200000e+03,2321.000000,1209.000000,5.640000e+02,3.194000e+03,3.194000e+03
mean,6.283467,29908.644578,3.106917e+07,54.336924,110.193548,3.490155e+07,4.400209e+07,8.534340e+07
std,1.252290,44937.582335,3.558591e+07,28.076593,20.171014,4.589512e+07,6.255531e+07,1.499473e+08
min,1.400000,18.000000,2.180000e+02,1.000000,46.000000,6.184540e+05,0.000000e+00,0.000000e+00
25%,5.600000,4828.500000,6.575000e+06,30.000000,95.000000,9.906211e+06,5.493221e+06,8.031285e+06
50%,6.400000,15106.000000,2.000000e+07,55.000000,107.000000,2.033156e+07,2.201947e+07,3.116893e+07
75%,7.200000,35810.500000,4.200000e+07,80.000000,121.000000,3.779422e+07,5.609176e+07,9.728380e+07
max,9.200000,519541.000000,3.000000e+08,100.000000,222.000000,3.525821e+08,7.601676e+08,2.767891e+09


In [118]:
cleaned = data.copy()
cleaned['Release_Date'] = cleaned.Release_Date.apply(parseDate).apply(lambda x: x.strftime('%Y-%m-%d') if isinstance(x, pd.Timestamp) else x)
cleaned.to_json('movies_clean.json', orient='records')

In [119]:
subset = data[["IMDB_Rating", "IMDB_Votes", "Rotten_Tomatoes_Rating", "Production_Budget",
               "Running_Time_min", "Release_Date", "US_DVD_Sales", "US_Gross", "Worldwide_Gross"]]

In [120]:
subset['Release_Date'] = data.Release_Date.apply(parseDate).apply(lambda x: x.date())

/Users/domoritz/anaconda3/envs/vega/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [121]:
subset['US_DVD_Sales'].fillna(0, inplace=True)

/Users/domoritz/anaconda3/envs/vega/lib/python3.6/site-packages/pandas/core/generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [122]:
subset.head()

,IMDB_Rating,IMDB_Votes,Rotten_Tomatoes_Rating,Production_Budget,Running_Time_min,Release_Date,US_DVD_Sales,US_Gross,Worldwide_Gross
0,6.1,1071.0,NaN,8000000.0,NaN,1998-06-12,0.0,146083.0,146083.0
1,6.9,207.0,NaN,300000.0,NaN,1998-08-07,0.0,10876.0,10876.0
2,6.8,865.0,NaN,250000.0,NaN,1998-08-28,0.0,203134.0,203134.0
3,NaN,NaN,13.0,300000.0,NaN,1998-09-11,0.0,373615.0,373615.0
4,3.4,165.0,62.0,1000000.0,NaN,1998-10-09,0.0,1009819.0,1087521.0


In [123]:
subset = subset.dropna()
len(subset)

982

In [124]:
table = pa.Table.from_pandas(subset)

writer = pa.RecordBatchFileWriter('movies.arrow', table.schema)
writer.write(table)
writer.close()

In [125]:
!ls -lah

total 5101208
drwxr-xr-x@ 32 domoritz  staff   1.0K Jul 10 00:21 .
drwxr-xr-x  22 domoritz  staff   704B Jul  9 23:53 ..
-rw-r--r--@  1 domoritz  staff   6.0K Nov 29  2017 .DS_Store
drwxr-xr-x   8 domoritz  staff   256B Jul  9 23:30 .ipynb_checkpoints
-rw-r--r--@  1 domoritz  staff    28M Nov 28  2017 564230852_T_ONTIME.csv
-rw-r--r--@  1 domoritz  staff   6.3M Nov 28  2017 564230852_T_ONTIME.zip
-rw-r--r--   1 domoritz  staff    12K Jul  9 15:38 convert_flights.ipynb
-rw-r--r--   1 domoritz  staff    16K Jul 10 00:21 convert_movies.ipynb
-rw-r--r--   1 domoritz  staff    17K Jul 10 00:03 convert_weather.ipynb
-rw-r--r--   1 domoritz  staff   276K Jun 29 14:51 flights-10k.arrow
-rw-r--r--   1 domoritz  staff   400K Nov 29  2017 flights-10k.csv
-rw-r--r--   1 domoritz  staff   409K Jun  8 22:39 flights-10k.json
-rw-r--r--   1 domoritz  staff   288M Jun 29 14:51 flights-10m.arrow
-rw-r--r--   1 domoritz  staff   439M Jun 16 10:21 flights-10m.csv
-rw-r--r--   1 domoritz  staff    29M Jun 